<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Why" data-toc-modified-id="Why-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Why</a></span></li><li><span><a href="#Doc" data-toc-modified-id="Doc-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Doc</a></span></li><li><span><a href="#Auth" data-toc-modified-id="Auth-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Auth</a></span></li><li><span><a href="#ORM" data-toc-modified-id="ORM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ORM</a></span><ul class="toc-item"><li><span><a href="#sheet" data-toc-modified-id="sheet-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>sheet</a></span></li><li><span><a href="#sheet---worksheet" data-toc-modified-id="sheet---worksheet-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>sheet - worksheet</a></span></li><li><span><a href="#worksheet" data-toc-modified-id="worksheet-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>worksheet</a></span></li></ul></li><li><span><a href="#Create" data-toc-modified-id="Create-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create</a></span><ul class="toc-item"><li><span><a href="#Sheet-object" data-toc-modified-id="Sheet-object-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Sheet object</a></span></li></ul></li><li><span><a href="#Update" data-toc-modified-id="Update-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Update</a></span></li><li><span><a href="#Read" data-toc-modified-id="Read-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Read</a></span></li><li><span><a href="#Delete" data-toc-modified-id="Delete-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Delete</a></span></li><li><span><a href="#Listen-a-cell" data-toc-modified-id="Listen-a-cell-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Listen a cell</a></span></li><li><span><a href="#Discussion" data-toc-modified-id="Discussion-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Discussion</a></span></li><li><span><a href="#Wrappers" data-toc-modified-id="Wrappers-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Wrappers</a></span><ul class="toc-item"><li><span><a href="#Some-test" data-toc-modified-id="Some-test-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Some test</a></span></li></ul></li></ul></div>

# Why

1. 更好的管理 sheets (**label**, eda shared with different team)
2. 每次都要從工作站下載 csv ，再貼上到 good drive，在複製貼上到 sheets 很麻煩

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['SPARK_HOME'] = '/opt/spark/versions/spark-3.1'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/opt/share/secrets/google-application-credential.json"

In [35]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
from os.path import join as PJ
from typing import List
import requests
import io

In [36]:
import pygsheets

In [37]:
pygsheets.__version__

'2.0.5'

# Doc

# Auth

In [21]:
# auth stuff

gc = pygsheets.authorize(
    service_account_file = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
)

# gc = pygsheets.authorize(service_account_env_var='GOOGLE_APPLICATION_CREDENTIALS')

In [22]:
print(dir(gc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'create', 'drive', 'get_range', 'logger', 'oauth', 'open', 'open_all', 'open_as_json', 'open_by_key', 'open_by_url', 'run_batch', 'set_batch_mode', 'sheet', 'spreadsheet_cls', 'spreadsheet_ids', 'spreadsheet_titles', 'teamDriveId']


# ORM

## sheet

In [53]:
# Case I - create a new sheet with service account
# It can do that! with some effert.

joe_sh = gc.create(title='Joe_Test')

#  A permission role casn only be one of ['organizer', 'owner', 'writer', 'commenter', 'reader'].
# also you can share a domain
joe_sh.share('joetsai@pixnet.tw',role='writer')

print('url of the sheets : ', joe_sh.url)
print('title of the sheets : ', joe_sh.title)

print('class and method : ')

print(joe_sh, dir(joe_sh), sep='\n\n')

# Findings :
# 1. the sheets create by the service account, so you might have no right to check the sheets
# 2. we might wanna add some useful information in __str__ e.g. url, Sheets name



url of the sheets :  https://docs.google.com/spreadsheets/d/1I7syWOOL6x89rpYrsVq2Lgr6DTZICz_g8F071OoBJ4k
title of the sheets :  Joe_Test
class and method : 
<Spreadsheet 'Joe_Test' Sheets:1>

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_defaultFormat', '_fetch_sheets', '_id', '_jsonsheet', '_named_ranges', '_sheet_list', '_title', 'add_worksheet', 'client', 'create_developer_metadata', 'custom_request', 'default_parse', 'defaultformat', 'del_worksheet', 'delete', 'export', 'fetch_properties', 'find', 'get_developer_metadata', 'id', 'logger', 'named_ranges', 'permissions', 'protected_ranges', 'remove_permission', 'replace', 'share', 'sheet1', 'title', 'to_json', '

## sheet - worksheet

In [54]:
sheet_by_id = gc.open_by_key(
    '1CBEjNnchIDzjU9uHdLh3GApNgO0AgdiyipppPnKMHd4'
)

sheet_by_id.worksheets()


[<Worksheet 'Sheet1' index:0>,
 <Worksheet '18d4f9c914fb4b679b097cf7ffc306fc' index:1>,
 <Worksheet '37b1fe247e0e477e8f0b4b7ee5b996c9' index:2>]

## worksheet

In [56]:
wks = sheet_by_id.worksheets()[0]

display(
    wks,
    wks.title,
    wks.__dict__,
    dir(wks)
)

<Worksheet 'Sheet1' index:0>

'Sheet1'

{'logger': <Logger pygsheets.worksheet (WARNING)>,
 'spreadsheet': <Spreadsheet 'Joe_Test' Sheets:3>,
 'client': <pygsheets.client.Client at 0x7fda6e3425b0>,
 '_linked': True,
 'jsonSheet': {'properties': {'sheetId': 0,
   'title': 'Sheet1',
   'index': 0,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 1000, 'columnCount': 26}}},
 'data_grid': None,
 '_func_calls': [],
 'grid_update_time': None}

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_func_calls',
 '_get_range',
 '_linked',
 '_update_grid',
 'add_chart',
 'add_cols',
 'add_conditional_formatting',
 'add_rows',
 'adjust_column_width',
 'adjust_row_height',
 'append_table',
 'cell',
 'clear',
 'client',
 'cols',
 'copy_to',
 'create_developer_metadata',
 'create_named_range',
 'create_protected_range',
 'data_grid',
 'delete_cols',
 'delete_named_range',
 'delete_rows',
 'export',
 'find',
 'frozen_cols',
 'frozen_rows',
 'get_all_records',
 'get_all_values',
 'get_as_df',
 'get_charts',
 'get_col',
 'get_developer_metadata',
 'get_gridrange',
 'get_named_range',
 'get_name

# Create

Create in drive should be easier

TODO : 

where is the sheets when service account create them?

## Sheet object

In [38]:
# Case I - create a new sheet with service account
# It can do that! with some effert.

joe_sh = gc.create(title='Joe_Test')

#  A permission role casn only be one of ['organizer', 'owner', 'writer', 'commenter', 'reader'].
# also you can share a domain
joe_sh.share('joetsai@pixnet.tw',role='writer')

print('url of the sheets : ', joe_sh.url)
print('title of the sheets : ', joe_sh.title)

# Findings :
# 1. the sheets create by the service account, so you might have no right to check the sheets
# 2. we might wanna add some useful information in __str__ e.g. url, Sheets name



url of the sheets :  https://docs.google.com/spreadsheets/d/1-y2v59ZMX3Ux91KSB7U7t-CwU9dI9wrumR3vuAN5uuo
title of the sheets :  Joe_Test
class and method : 
<Spreadsheet 'Joe_Test' Sheets:1>

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_defaultFormat', '_fetch_sheets', '_id', '_jsonsheet', '_named_ranges', '_sheet_list', '_title', 'add_worksheet', 'client', 'create_developer_metadata', 'custom_request', 'default_parse', 'defaultformat', 'del_worksheet', 'delete', 'export', 'fetch_properties', 'find', 'get_developer_metadata', 'id', 'logger', 'named_ranges', 'permissions', 'protected_ranges', 'remove_permission', 'replace', 'share', 'sheet1', 'title', 'to_json', '

In [40]:
# Case II - create a new worksheet with a given sheet

import uuid

random_name = uuid.uuid4().hex
print(random_name)

sample_pdf = pd.DataFrame(
{
    'a': np.random.randint(low=0,high=1000,size=1000),
    'b': np.random.randint(low=0,high=1000,size=1000)
})

default_n_rows = 1000
default_n_cols = 100

wks = joe_sh.add_worksheet(random_name)
wks.set_dataframe(sample_pdf,(0,0))
wks_share_url = f"https://docs.google.com/spreadsheets/d/{joe_sh.id}/edit#gid={wks.id}"
print(wks_share_url)

e2d89946fa684928951d9571e82a90f6
https://docs.google.com/spreadsheets/d/1-y2v59ZMX3Ux91KSB7U7t-CwU9dI9wrumR3vuAN5uuo/edit#gid=1051194199


# Update

In [41]:
# Open spreadsheet and then worksheet

# return a sheets
# need to setup the crenditial
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=0')

print(type(sh), sh)

# working sheets

wks = sh.sheet1

# Update a cell with value (just to let him know values is updated ;) )
wks.update_value('A1', "Hey yank this numpy array")
my_nparray = np.random.randint(10, size=(3, 4))

# update the sheet with array
wks.update_values('A2', my_nparray.tolist())

# already update the value =)

# https://docs.google.com/spreadsheets/d/1KLzEsuFdx2_9o1Kf7GgR_S-iBQQ20ZDHWpMClKDVtWE/edit#gid=0

<class 'pygsheets.spreadsheet.Spreadsheet'> <Spreadsheet 'test' Sheets:2>


In [42]:
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=847289509')

In [44]:
df = (
    pd.DataFrame(
    {
        'note':['','',''],
        'label':['','',''],
        'string_id':['0001','0002','0003'],
        'aonther_id':['1','2','3'],
        'poi_name' : ['abc','defg','sss'],
        'views' : [100, 50, np.nan],
        'click' : [3, 1, 0]
    }
    )
)

try:
    wks = sh.worksheet_by_title('poi_views_v20210804')
except:
    wks = sh.add_worksheet('poi_views_v20210804')
print('Now we have : ',sh.worksheets(), wks, sep='\n\n')

# write dataframe into sheets
wks.set_dataframe(df, start=(0,0), nan='NotaNumber')

print(sh, wks, sh.url, sep='\n\n')
# Findins:

# 1. write dataframe into sheets with ease
# 2. need a work-around about creating sheets
# 3. we might wanna add a sanity check if we wanna wrtie dataframe into a sheet
#    avoiding overwrite the already-labelled data

Now we have : 

[<Worksheet 'example_1' index:0>, <Worksheet '工作表3' index:1>, <Worksheet 'poi_views_v20210804' index:2>]

<Worksheet 'poi_views_v20210804' index:2>
<Spreadsheet 'test' Sheets:3>

<Worksheet 'poi_views_v20210804' index:2>

https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   note        3 non-null      object 
 1   label       3 non-null      object 
 2   string_id   3 non-null      object 
 3   aonther_id  3 non-null      object 
 4   poi_name    3 non-null      object 
 5   views       2 non-null      float64
 6   click       3 non-null      object 
dtypes: float64(1), object(6)
memory usage: 296.0+ bytes


# Read

In [49]:
wks

<Worksheet 'poi_views_v20210804' index:2>

In [50]:
# you can also do export(csv) and read
# wks.export(xxx.csv), pd_read_csv(xxx.csv)

labelled_df = wks.get_as_df(
    numerize=False # watch out the 0001 will be conver to 1
)

labelled_df.info()
labelled_df

# Findings:
# 1. read sheet into dataframe with ease
# 2. dtype might be different when convert sheets back to dataframe (e.g. 0001 --> 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   note        3 non-null      object
 1   label       3 non-null      object
 2   string_id   3 non-null      object
 3   aonther_id  3 non-null      object
 4   poi_name    3 non-null      object
 5   views       3 non-null      object
 6   click       3 non-null      object
dtypes: object(7)
memory usage: 296.0+ bytes


,note,label,string_id,aonther_id,poi_name,views,click
0,,,1,1,abc,100,3
1,,,2,2,defg,50,1
2,,,3,3,sss,NotaNumber,0


In [51]:
# read multiple 

for worksheet in sh.worksheets():
    labelled_df = worksheet.get_as_df(
    numerize=False # watch out the 0001 will be conver to 1
    )
    display(
        worksheet.title,
        worksheet.id,
        labelled_df
    )


'example_1'

1362269438

,Hey yank this numpy array,label,string_id,another_id,poi_name,views,click
0,9,0,0,7,abc,100,3
1,9,5,6,4,defg,50,1
2,8,2,1,5,sss,,0


'工作表3'

1810733077

,note,label,string_id,another_id,poi_name,views,click
0,1,1,1,5,abc,100,3
1,1,1,1,0,defg,50,1
2,1,1,1,7,sss,,0


'poi_views_v20210804'

567298347

,note,label,string_id,aonther_id,poi_name,views,click
0,,,1,1,abc,100,3
1,,,2,2,defg,50,1
2,,,3,3,sss,NotaNumber,0


# Delete

In [22]:
sh.del_worksheet(wks)
# Findings : 
# 1. be careful, it will delete the sheets directly, not a safe-delete(e.g. move into the trash-can)
# 2. we might wanna write a wrapper about safe-delete

# Listen a cell 

In [23]:
pass

# Discussion


1. `pygsheets` could be a prototype to helps us manage the sheets


2. Use case : 

    - [x] 經常使用 sheets 跨團隊協作，例如共同資料探索，收取 Domain Expert 意見、收取 Label 等
    - [ ] 監聽某個 cell - 作為 trigger
    - [ ] 

3. Meta data(sheets url as data source)

    - 同個 projects 每一次的 label data, schema 可能不太一樣 --> poi_article_mapping 每一次做資料驗證時，關注的重點可能皆不同(poi_name, article_title, or poi_hub, article_for_dummies) --> schema free?

# Wrappers

In [24]:
def read_spreadsheet_to_pandas_v0(sheet_id : str,
                                  gid : int = 0,
                                  sheet_view : bool = False,
                                  fillna=None,
                                  **kwargs):
    """Download google spreadsheet as pandas.DataFrame
    Args:
        sheet_id: The hash id in a google spreadsheet's url. (e.g. 1evuGqCk3B7VWez1JAw249YjIG3rKznDFHz8I_0A64KY)
        fillna: The value that will replace NaN values in the spreadsheet, default is None.
        **kwargs: Users can add settings for pandas to read csv into dataframe. (e.g. pandas's schema dtype=dtype)"""

    download_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
    
    if sheet_view:
        print(f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit#gid={gid}")
    
    s = requests.get(download_url).content
    df = pd.read_csv(io.StringIO(s.decode("utf-8")), **kwargs)

    if fillna is not None:
        df = df.fillna(fillna)

    return df


In [25]:
# Read sheets as dataframe
# Input sheets_id, gid
# Remember to do auth stuff

# gc = pygsheets.authorize(service_account_file='/opt/share/secrets/google-application-credential.json')


from pixlake.catadog.models.spreadsheet import read_spreadsheet_to_pandas as read_spreadsheet_to_pandas_v1

In [28]:
# sometimes work, sometimes dosn't work
res = read_spreadsheet_to_pandas_v0(sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
                                    gid=1362269438,
                                    sheet_view=True
                                   )

# https://docs.google.com/spreadsheets/d/1NLeYxjm5rKtMb90sNgHggDuA1B_Lrc4m_wPJJ0LskhI/export?format=csv&gid=0
try:
    res2 = read_spreadsheet_to_pandas_v0(sheet_id='1NLeYxjm5rKtMb90sNgHggDuA1B_Lrc4m_wPJJ0LskhI',
                                gid=0,
                                sheet_view=True
                               )
except Exception as e:
    print(e)

https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438
https://docs.google.com/spreadsheets/d/1NLeYxjm5rKtMb90sNgHggDuA1B_Lrc4m_wPJJ0LskhI/edit#gid=0
Error tokenizing data. C error: Expected 1 fields in line 6, saw 2



In [29]:
# more stable

res = read_spreadsheet_to_pandas_v1(sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
                                    gid=1362269438,
                                    sheet_view=True
                                   )

# https://docs.google.com/spreadsheets/d/1NLeYxjm5rKtMb90sNgHggDuA1B_Lrc4m_wPJJ0LskhI/export?format=csv&gid=0
    
res2 = read_spreadsheet_to_pandas_v1(sheet_id='1NLeYxjm5rKtMb90sNgHggDuA1B_Lrc4m_wPJJ0LskhI',
                                gid=0,
                                sheet_view=True
                               )

https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438
https://docs.google.com/spreadsheets/d/1NLeYxjm5rKtMb90sNgHggDuA1B_Lrc4m_wPJJ0LskhI/edit#gid=0


In [30]:
display(res, res2)
type(res)

,Hey yank this numpy array,label,string_id,another_id,poi_name,views,click
0,8,2,8,5,abc,100,3
1,4,2,4,0,defg,50,1
2,9,7,7,7,sss,,0


,address_city,address_street,address_road_norm,address_city_norm,POI_name,commerical_area,label,reason
0,南投縣,中正路572號,中正路,南投縣,明園川菜餐廳,草屯鎮形象商圈,,
1,南投縣,中正路98號,中正路,南投縣,鹿港蚵嗲,在地商圈,,
2,南投縣,中興路247號,中興路,南投縣,美食堂,在地商圈,,
3,南投縣,中正路23號2樓,中正路,南投縣,雲品酒店丹彤餐廳,埔里鎮形象商圈,,
4,南投縣,橫街27號,橫街,南投縣,車站前蛋餅,竹山商圈,,
...,...,...,...,...,...,...,...,...
205,高雄市,中山路485號,中山路,高雄市,鍋堂,旗山老街,,
206,高雄市,文化路87號,文化路,高雄市,阿德牛排,新堀江商圈,,
207,高雄市,中華三路61號,中華三路,高雄市,麻辣王朝,三鳳中街,,
208,高雄市,文橫二路28號,文橫二路,高雄市,怎樣一杯杯新光店,興中觀光商圈,,


pandas.core.frame.DataFrame

In [27]:
from pixlake.catadog.models.spreadsheet import Spreadsheet, Worksheet

In [7]:
wks1=Worksheet(
    sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
    gid = 1362269438,
    description = """label data with poi_article pairs,
    without article_with_dummies, poi_hub""",
    schema={
        'note' : str,
        'label' : str,
        'string_id' : str,
        'another_id' : int,
        'poi_name' : str,
        "views" : float,
        "click" : float
            }
)

In [8]:
wks1

Worksheet(description='label data with poi_article pairs,\n    without article_with_dummies, poi_hub', schema={'note': <class 'str'>, 'label': <class 'str'>, 'string_id': <class 'str'>, 'another_id': <class 'int'>, 'poi_name': <class 'str'>, 'views': <class 'float'>, 'click': <class 'float'>}, sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A', gid=1362269438, identifier='')

In [9]:
wks1.share_url

'https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438'

In [10]:
pdf = wks1.load()

pdf

,note,label,string_id,another_id,poi_name,views,click
0,8,2,8,5,abc,100,3.0
1,4,2,4,0,defg,50,1.0
2,9,7,7,7,sss,,0.0


In [11]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   note        3 non-null      object 
 1   label       3 non-null      object 
 2   string_id   3 non-null      object 
 3   another_id  3 non-null      int64  
 4   poi_name    3 non-null      object 
 5   views       3 non-null      object 
 6   click       3 non-null      float64
dtypes: float64(1), int64(1), object(5)
memory usage: 296.0+ bytes


In [12]:
sdf = wks1.load(load_type="spark")

sdf.printSchema()

sdf.show()

21/09/24 08:27:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


root
 |-- note: string (nullable = true)
 |-- label: string (nullable = true)
 |-- string_id: string (nullable = true)
 |-- another_id: long (nullable = true)
 |-- poi_name: string (nullable = true)
 |-- views: string (nullable = true)
 |-- click: double (nullable = true)



+----+-----+---------+----------+--------+-----+-----+
|note|label|string_id|another_id|poi_name|views|click|
+----+-----+---------+----------+--------+-----+-----+
|   8|    2|        8|         5|     abc|  100|  3.0|
|   4|    2|        4|         0|    defg|   50|  1.0|
|   9|    7|        7|         7|     sss|     |  0.0|
+----+-----+---------+----------+--------+-----+-----+



In [13]:
wks1.dump(pdf.fillna(''), mode='overwrite')

Already export dataframe to :  https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438


Worksheet(description='label data with poi_article pairs,\n    without article_with_dummies, poi_hub', schema={'note': <class 'str'>, 'label': <class 'str'>, 'string_id': <class 'str'>, 'another_id': <class 'int'>, 'poi_name': <class 'str'>, 'views': <class 'float'>, 'click': <class 'float'>}, sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A', gid=1362269438, identifier='')

In [28]:
s2=Spreadsheet(
    sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
    description = """label data with poi_article pairs,
    without article_with_dummies, poi_hub""",
)

s2.share_url

# pairs = s2.load()

'https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=0'

In [29]:
# sheet = gc.open_by_key("1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A")
# r = {}
# for wks in sheet.worksheets():
#     r[wks.title] = 

[<Worksheet 'example_1' index:0>, <Worksheet '工作表3' index:1>]

In [ ]:
pairs = s2.load()

In [ ]:
# s2=Spreadsheet(
#     sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
#     description = """label data with poi_article pairs,
#     without article_with_dummies, poi_hub""",
#     schema={
#         'note' : str,
#         'label' : str,
#         'string_id' : str,
#         'another_id' : int,
#         'poi_name' : str,
#         "views" : float,
#         "click" : float
#             }
# )


# sheet_content_pair = s2.load_all()

# sheet_content_pair_sdf = s2.load_all(load_type="spark")

In [ ]:
for name, pdf in sheet_content_pair.items():
    display(name, pdf)
    
for name, sdf in sheet_content_pair_sdf.items():
    print(name)
    print()
    sdf.show()

## Some test

In [ ]:
a = 0 or None
a

In [ ]:
a = 435 or None
a

In [ ]:
a = None or 45
a

In [ ]:
a = None or 0

a